In [1]:
from keras.models import load_model
import cv2
import numpy as np
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
import email 
import smtplib    
from playsound import playsound
from email.message import EmailMessage
import base64

In [2]:
model = load_model('models/model-008.model')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [3]:
no_of_Beeps = 0
while(True):

    ret,img=source.read()
    print(img.shape)
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for x,y,w,h in faces:
        global label

        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label = np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    def sendEmail():
        # try:
        msg = MIMEMultipart()
        msg['From'] = 'demoemailsweety@gmail.com'
        msg['To'] = 'demoemailsweety@gmail.com'
        msg['Subject'] = 'No mask found'
        msg_content = MIMEText('No mask found', 'plain', 'utf-8')
        msg.attach(msg_content)
        with open("C:/Users/dhrav/Python Projects/Mask detector/capturedImg/no_mask.png", 'rb') as f:
            # set attachment mime and file name, the image type is png
            mime = MIMEBase('image', 'png', filename='no_mask.png')
            # add required header data:
            mime.add_header('Content-Disposition', 'attachment', filename='no_mask.png')
            mime.add_header('X-Attachment-Id', '0')
            mime.add_header('Content-ID', '<0>')
            # read attachment file content into the MIMEBase object
            mime.set_payload(f.read())
            # encode with base64
            email.encoders.encode_base64(mime)
            
            # add MIMEBase object to MIMEMultipart object
            msg.attach(mime)
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.ehlo()
        server.starttls()
        server.login('demoemailsweety@gmail.com', 'DEMO@123')
        server.sendmail('demoemailsweety@gmail.com', 'demoemailsweety@gmail.com', msg.as_string())
        server.close()
        # except Exception as e:
        #     pass
    try:
        if label == 1:
            playsound('beep.mp3')
            no_of_Beeps+= 1
            print(no_of_Beeps)

        if no_of_Beeps%5==0 and no_of_Beeps !=0:
            cv2.imwrite('C:/Users/dhrav/Python Projects/Mask detector/capturedImg/no_mask.png',img)
            # sendEmail()
    except Exception as e :
        print(e)

    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)

    if(key==27):
        break
cv2.destroyAllWindows()
source.release()

(480, 640, 3)


error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-m8us58q4\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'
